In [1]:
# all neccessary libraries
from nltk.corpus import stopwords
#stopwords : a an the etc.
from nltk.cluster.util import cosine_distance
#used to build similarity matrix
import numpy as np
import networkx as nx
#library to study graphs and networks

In [2]:
import nltk

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ad_demon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Generate sentences from text
def read_text(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
#     print(filedata)
    article = filedata[0].split(". ")
#     print(article)
    sentences = []
    for sentence in article:
#         print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        #[^a-zA-Z] means any character that is not in a to z or A to Z
    sentences.pop()
    return sentences

In [5]:
def sentence_similarity(sent1, sent2, stopwords=None):
#     print(sent1,'s1', sent2, 's2')
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
#     print(len(sent1)+len(sent2))
    all_words = list(set(sent1 + sent2))
#     print(len(all_words))
    # forming list of unique words from the 2 lists
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
    # removing meaningless words like a, an , the by keeping there vector value 0
    #we are increasing vector value of non stop words
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [6]:
#Building similarity matrix from the sentences
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    # zero matrix
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

In [9]:
def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
#     print(len(stop_words))
    # Step 1 - Read text and tokenize
    sentences =  read_text(file_name)
    print(len(sentences))
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)
#     print(sentence_similarity_matrix)
    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    #nx.from_numpy_array returns a graph from a 2d numpy array
#     print(sentence_similarity_graph)
    scores = nx.pagerank(sentence_similarity_graph)
    print(scores)
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
        # Step 5 - Offcourse, output the summarize text
    print('\n\n')
    print("Summarize Text: \n", ". ".join(summarize_text))

In [10]:
generate_summary("sample.txt", 10)

26
{0: 0.07499550456599864, 1: 0.07928056792388954, 2: 0.015150679603697266, 3: 0.053943336509824684, 4: 0.04155487734551386, 5: 0.031395096945447454, 6: 0.006172839506172842, 7: 0.05036133172428871, 8: 0.010181205690460642, 9: 0.025769768504439548, 10: 0.04672195035383562, 11: 0.027713065149188084, 12: 0.06558775623320807, 13: 0.028957773909222998, 14: 0.059085613808469634, 15: 0.053126786391878754, 16: 0.02700767821333785, 17: 0.037839369849017715, 18: 0.04982245853188474, 19: 0.06826209136314952, 20: 0.03362527416403969, 21: 0.039010769463216535, 22: 0.006172839506172842, 23: 0.01689889937618024, 24: 0.026980737557647853, 25: 0.02438172780981673}
Indexes of top ranked_sentence order are  [(0.07928056792388954, ['Modi', 'was', 'the', 'chief', 'minister', 'of', 'Gujarat', 'from', '2001', 'to', '2014', 'and', 'is', 'a', 'Member', 'of', 'Parliament', 'from', 'Varanasi']), (0.07499550456599864, ['Narendra', 'Damodardas', 'Modi', '(', 'born', '17', 'September', '1950)', 'is', 'an', 'India